In [10]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tqdm import tqdm
import matplotlib.pyplot as plt
import pathlib
import os

AUTOTUNE = tf.data.experimental.AUTOTUNE
tf.keras.backend.clear_session()  # For easy reset of notebook state.

tf.__version__
tf.executing_eagerly()

True

In [11]:

ds = tfds.load(name="imagenette", with_info=True, split=["train[:1000]","validation[:100]"])
ds_train=ds[0][0]
ds_test=ds[0][1]
assert isinstance(ds_train, tf.data.Dataset)

def normalize(x):
    y = {'image': tf.image.convert_image_dtype(x['image'], tf.float32), 'label': x['label']}
    y = (tf.image.resize(y['image'], (224,224)), y['label'])
    return y
    
ds_train = ds_train.map(lambda x: normalize(x))
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(100)
ds_train = ds_train.batch(8)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(8)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [12]:
weights = {
    'weights_conv_1': tf.Variable(tf.random.normal([3, 3, 3, 8])),
    'weights_conv_2': tf.Variable(tf.random.normal([3, 3, 8, 12])),
    'weights_conv_3': tf.Variable(tf.random.normal([3, 3, 12, 16])),
    'weights_conv_4': tf.Variable(tf.random.normal([3, 3, 128, 128])),
    'weights_conv_5': tf.Variable(tf.random.normal([3, 3, 128, 256])),
    'weights_conv_6': tf.Variable(tf.random.normal([3, 3, 256, 256])),
    'weights_conv_7': tf.Variable(tf.random.normal([3, 3, 256, 256])),
    'weights_conv_8': tf.Variable(tf.random.normal([3, 3, 256, 512])),
    'weights_conv_9': tf.Variable(tf.random.normal([3, 3, 512, 512])),
    'weights_conv_10': tf.Variable(tf.random.normal([3, 3, 512, 512])),
    'weights_conv_11': tf.Variable(tf.random.normal([3, 3, 512, 512])),
    'weights_conv_12': tf.Variable(tf.random.normal([3, 3, 512, 512])),
    'weights_conv_13': tf.Variable(tf.random.normal([3, 3, 512, 512])),

    'weights_dense_1': tf.Variable(tf.random.normal([28*28*16, 4096])),
    'weights_dense_2': tf.Variable(tf.random.normal([4096, 4096])),
    'weights_dense_3': tf.Variable(tf.random.normal([4096, 10])),
}

masks = {
    
    'mask_conv_1': tf.Variable(tf.ones([3, 3, 3, 8]), trainable=False),
    'mask_conv_2': tf.Variable(tf.ones([3, 3, 8, 12]), trainable=False),
    'mask_conv_3': tf.Variable(tf.ones([3, 3, 12, 16]), trainable=False),
    'mask_conv_4': tf.Variable(tf.ones([3, 3, 128, 128]), trainable=False),
    'mask_conv_5': tf.Variable(tf.ones([3, 3, 128, 256]), trainable=False),
    'mask_conv_6': tf.Variable(tf.ones([3, 3, 256, 256]), trainable=False),
    'mask_conv_7': tf.Variable(tf.ones([3, 3, 256, 256]), trainable=False),
    'mask_conv_8': tf.Variable(tf.ones([3, 3, 256, 512]), trainable=False),
    'mask_conv_9': tf.Variable(tf.ones([3, 3, 512, 512]), trainable=False),
    'mask_conv_10': tf.Variable(tf.ones([3, 3, 512, 512]), trainable=False),
    'mask_conv_11': tf.Variable(tf.ones([3, 3, 512, 512]), trainable=False),
    'mask_conv_12': tf.Variable(tf.ones([3, 3, 512, 512]), trainable=False),
    'mask_conv_13': tf.Variable(tf.ones([3, 3, 512, 512]), trainable=False),
    # 224x224 input --> 5 maxpool layers --> 
    'mask_dense_1': tf.Variable(tf.ones([28*28*16, 4096]), trainable=False),
    'mask_dense_2': tf.Variable(tf.ones([4096, 4096]), trainable=False),
    'mask_dense_3': tf.Variable(tf.ones([4096, 10]), trainable=False),
}

biases = {
    #output depth
    'bias_conv_1': tf.Variable(tf.random.normal([8])),
    'bias_conv_2': tf.Variable(tf.random.normal([12])),
    'bias_conv_3': tf.Variable(tf.random.normal([16])),
    'bias_conv_4': tf.Variable(tf.random.normal([128])),
    'bias_conv_5': tf.Variable(tf.random.normal([256])),
    'bias_conv_6': tf.Variable(tf.random.normal([256])),
    'bias_conv_7': tf.Variable(tf.random.normal([256])),
    'bias_conv_8': tf.Variable(tf.random.normal([512])),
    'bias_conv_9': tf.Variable(tf.random.normal([512])),
    'bias_conv_10': tf.Variable(tf.random.normal([512])),
    'bias_conv_11': tf.Variable(tf.random.normal([512])),
    'bias_conv_12': tf.Variable(tf.random.normal([512])),
    'bias_conv_13': tf.Variable(tf.random.normal([512])),
    
    'bias_dense_1': tf.Variable(tf.random.normal([4096])),
    'bias_dense_2': tf.Variable(tf.random.normal([4096])),
    'bias_dense_3': tf.Variable(tf.random.normal([10])),

}


ResourceExhaustedError: OOM when allocating tensor with shape[12544,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomStandardNormal]

In [ ]:
#conv2D with bias and relu activation

class CustomConvLayer(layers.Layer):

    def __init__(self, weights, mask, biases, strides, padding='SAME'):
        
        super(CustomConvLayer, self).__init__()
        self.w = weights
        self.m = mask
        self.b = biases
        self.s = strides
        self.p = padding

        
    def call(self, inputs):
        x = tf.nn.conv2d(inputs, tf.multiply(self.w, self.m), strides=[1, self.s, self.s, 1], padding=self.p)
        x = tf.nn.bias_add(x, self.b)
        return tf.nn.relu(x)
        

#Average Pooling Layer
class CustomPoolLayer(layers.Layer):
    
    def __init__(self, k=2, padding='SAME'):#padding='VALID'):
        super(CustomPoolLayer, self).__init__()
        self.k = k
        self.p = padding
    
    def call(self, inputs):
        return tf.nn.max_pool2d(inputs, ksize=[1, self.k, self.k,1], strides=[1, self.k, self.k, 1], padding=self.p)
    
#Dense Layer with Bias
class CustomDenseLayer(layers.Layer):
    
    def __init__(self, weights, mask, bias, activation = 'relu'):
        super(CustomDenseLayer, self).__init__()
        self.w = weights
        self.b = bias
        self.a = activation
        self.m = mask
        
    def call(self, inputs):
        #print('dense w',self.w)
        #print('dense i',inputs)
        x = tf.matmul(inputs, tf.multiply(self.w, self.m))
        #print('dense x',x)
        x = tf.nn.bias_add(x, self.b)
        if self.a == 'relu':
            return tf.nn.relu(x)
        if self.a == 'softmax':
            return tf.nn.softmax(x)
        if self.a == 'sigmoid':
            return tf.nn.sigmoid(x)

In [ ]:
class VGG16(tf.keras.Model):
    def __init__(self):
        super(VGG16, self).__init__()
        self.conv1 = CustomConvLayer(weights['weights_conv_1'], masks['mask_conv_1'], biases['bias_conv_1'], 1)
        self.bn1 = layers.BatchNormalization()
        self.maxpool2 = CustomPoolLayer(k=2)
        self.conv2 = CustomConvLayer(weights['weights_conv_2'], masks['mask_conv_2'], biases['bias_conv_2'], 1)
        self.bn2 = layers.BatchNormalization()
        self.maxpool1 = CustomPoolLayer(k=2)
        self.conv3 = CustomConvLayer(weights['weights_conv_3'], masks['mask_conv_3'], biases['bias_conv_3'], 1)
        self.bn3 = layers.BatchNormalization()
        #self.conv4 = CustomConvLayer(weights['weights_conv_4'], masks['mask_conv_4'], biases['bias_conv_4'], 1)
        
        #self.conv5 = CustomConvLayer(weights['weights_conv_5'], masks['mask_conv_5'], biases['bias_conv_5'], 1)
        #self.conv6 = CustomConvLayer(weights['weights_conv_6'], masks['mask_conv_6'], biases['bias_conv_6'], 1)
        #self.conv7 = CustomConvLayer(weights['weights_conv_7'], masks['mask_conv_7'], biases['bias_conv_7'], 1)
        #self.maxpool3 = CustomPoolLayer(k=2)
        #self.conv8 = CustomConvLayer(weights['weights_conv_8'], masks['mask_conv_8'], biases['bias_conv_8'], 1)
        #self.conv9 = CustomConvLayer(weights['weights_conv_9'], masks['mask_conv_9'], biases['bias_conv_9'], 1)
        #self.conv10 = CustomConvLayer(weights['weights_conv_10'], masks['mask_conv_10'], biases['bias_conv_10'], 1)
        self.maxpool4 = CustomPoolLayer(k=2)
        #self.conv11 = CustomConvLayer(weights['weights_conv_11'], masks['mask_conv_11'], biases['bias_conv_11'], 1)
        #self.conv12 = CustomConvLayer(weights['weights_conv_12'], masks['mask_conv_12'], biases['bias_conv_12'], 1)
        #self.conv13 = CustomConvLayer(weights['weights_conv_13'], masks['mask_conv_13'], biases['bias_conv_13'], 1)
        #self.maxpool5 = CustomPoolLayer(k=2)
        self.dense1 = CustomDenseLayer(weights['weights_dense_1'], masks['mask_dense_1'], biases['bias_dense_1'], 'relu')
        self.dense2 = CustomDenseLayer(weights['weights_dense_2'], masks['mask_dense_2'], biases['bias_dense_2'], 'relu')
        self.dense3 = CustomDenseLayer(weights['weights_dense_3'], masks['mask_dense_3'], biases['bias_dense_3'], 'softmax')
        
    def call(self, inputs, training=False):
        #x = tf.reshape(inputs, shape=[-1, 28, 28, 1])
        print(inputs.shape)
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.maxpool1(x)
        print(x.shape)
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.bn3(x, training=training)
        #x = self.conv4(x)
        print(x.shape)
        #x = self.conv5(x)
        #x = self.conv6(x)
        #x = self.conv7(x)
        #x = self.maxpool3(x)
        #x = self.conv8(x)
        #x = self.conv9(x)
        #x = self.conv10(x)
        #x = self.maxpool4(x)
        #x = self.conv11(x)
        #x = self.conv12(x)
        #x = self.conv13(x)
        x = self.maxpool4(x)
        print(x.shape)
        x = layers.Flatten()(x)
        print(x.shape)
        x =  self.dense1(x)
        x =  self.dense2(x)
        x =  self.dense3(x)
        return x
        

In [ ]:
model = VGG16()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'],
              experimental_run_tf_function=False
             )

In [ ]:
model.fit(
    x=ds_train,
    #steps_per_epoch=1,
    epochs=7,
    validation_data=ds_test,
)

In [69]:
model.fit(
    x=ds_train,
    #steps_per_epoch=1,
    epochs=7,
    validation_data=ds_test,
)

Epoch 1/7
    125/Unknown - 74s 595ms/step - loss: 2.3422 - accuracy: 0.1190Epoch 1/7
125/125 [==============================] - 76s 612ms/step - loss: 2.3431 - accuracy: 0.1190 - val_loss: 2.3362 - val_accuracy: 0.1300
Epoch 2/7
125/125 [==============================] - 79s 628ms/step - loss: 2.3432 - accuracy: 0.1180 - val_loss: 2.3362 - val_accuracy: 0.1300
Epoch 3/7
125/125 [==============================] - 78s 620ms/step - loss: 2.3482 - accuracy: 0.1130 - val_loss: 2.3362 - val_accuracy: 0.1300
Epoch 4/7
125/125 [==============================] - 95s 761ms/step - loss: 2.3472 - accuracy: 0.1140 - val_loss: 2.3362 - val_accuracy: 0.1300
Epoch 5/7
 91/125 [====================>.........] - ETA: 32s - loss: 2.3444 - accuracy: 0.1168

KeyboardInterrupt: 

In [ ]:
def create_data_set(PATH):
    data_dir = pathlib.Path(PATH)
    CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if (item.name != "LICENSE.txt" and item.name != '.DS_Store')])
    image_count = len(list(data_dir.glob('*/*.JPEG')))
    IMG_HEIGHT = 224
dd    IMG_WIDTH = 224
    
    def get_label(file_path):
        # convert the path to a list of path components
        
        parts = tf.strings.split(file_path, os.path.sep)
        # The second to last is the class-directory
        return parts[-2] == CLASS_NAMES

    def decode_img(img):
        # convert the compressed string to a 3D uint8 tensor
        img = tf.image.decode_jpeg(img, channels=3)
        # Use `convert_image_dtype` to convert to floats in the [0,1] range.
        img = tf.image.convert_image_dtype(img, tf.float32)
        # resize the image to the desired size.
        img = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])
        #print(img)
        return img

    def process_path(file_path):
        label = get_label(file_path)
        # load the raw data from the file as a string
        img = tf.io.read_file(file_path)
        #print(img)
        img = decode_img(img)
        #print(type(img))
        return img, label
    
    list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*.JPEG'))
    print(list(list_ds))
    labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
    return labeled_ds
    

In [ ]:
TRAIN_PATH = '/Users/florianmerkle/tensorflow_datasets/downloads/extracted/TAR_GZ.s3_fast-ai-imageclas_imagenetteG9ZQvBaITKiOTw9TfthWmx-Neuhl0366js3YfZzZ3Po.tgz/imagenette/train'
train_data = create_data_set(TRAIN_PATH)
VAL_PATH = '/Users/florianmerkle/tensorflow_datasets/downloads/extracted/TAR_GZ.s3_fast-ai-imageclas_imagenetteG9ZQvBaITKiOTw9TfthWmx-Neuhl0366js3YfZzZ3Po.tgz/imagenette/val/'
val_data = create_data_set(VAL_PATH)

In [ ]:
def prepare_for_training(ds, cache=False, shuffle_buffer_size=1000):
  # This is a small dataset, only load it once, and keep it in memory.
  # use `.cache(filename)` to cache preprocessing work for datasets that don't
  # fit in memory.
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    #ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat forever
    ds = ds.repeat(1)

    ds = ds.batch(BATCH_SIZE)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds

In [ ]:
BATCH_SIZE = 64
train_ds = prepare_for_training(train_data)
val_ds = prepare_for_training(val_data)

In [ ]:
x_train = list(val_ds)[0][0]
y_train = list(val_ds)[0][1]
#x_train = tf.expand_dims(x_train, axis=0)
#y_train = tf.expand_dims(y_train, axis=0)
print(y_train[0].shape)

In [ ]:
model.fit(x=x_train,
          y=labels,
          epochs=5,
          batch_size=4
         )

In [ ]:
model.run_eagerly = True

In [ ]:
y_train

In [ ]:
#model.predict_on_batch(val_ds)
labels = np.asarray([np.argmax(x)for x in y_train])

In [ ]:
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        #print()
        plt.title(CLASS_NAMES[np.argmax(label_batch[n])])
        plt.axis('off')
        

In [ ]:
image_batch, label_batch = next(iter(train_ds))

#show_batch(x_train[:25], y_train[:25])
show_batch(image_batch, label_batch)

In [ ]:
CLASS_NAMES

In [ ]:
data_dir = pathlib.Path(TRAIN_PATH)
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if (item.name != "LICENSE.txt" and item.name != '.DS_Store')])
image_count = len(list(data_dir.glob('**/*.JPEG')))

In [ ]:
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)
STEPS_PER_EPOCH

In [ ]:
image_count

In [ ]:
IMG_PATH = '/Users/florianmerkle/tensorflow_datasets/downloads/extracted/TAR_GZ.s3_fast-ai-imageclas_imagenetteG9ZQvBaITKiOTw9TfthWmx-Neuhl0366js3YfZzZ3Po.tgz/imagenette/train/chain_saw/n03000684_1000.JPEG'
def predict():
    image = tf.io.read_file(IMG_PATH)
    image = tf.image.decode_jpeg(image, channels=3)
    print(image.shape)
    image = tf.expand_dims(image, axis=0)
    print(image.shape)
    image = tf.image.resize(image, [224, 224])
    print(image.shape)
    print(model.predict(image))

In [ ]:
vgg_keras = tf.keras.applications.VGG16()

vgg_keras.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True) ,
              metrics=['accuracy'],
              experimental_run_tf_function=False
             )

In [ ]:
preds = vgg_keras.predict(first_b[0][:10])

In [ ]:
model.summary()